<i><b>Public-AI</b></i>

#### week 1. 고객 데이터와 개인화 추천시스템

# Section 2. 아이템 간 연관 관계를 활용한 추천

앞서 Section 1에서는 데이터의 기본적인 통계치에 따라 정렬하여 추천 할 상품을 정하는 비개인화 추천 방식을 배웠습니다. 이번에는 아이템 간의 연관관계를 수치화 해보고, 이를 활용한 추천시스템을 만들어보겠습니다. 


### _Objective_ 

* [아이템 간 관계를 나타내는 여러가지 지표 ] 아이템의 연관 관계를 파악하는 세가지 지표 (Support, Confidence, Lift)를 살펴봅니다.
* [빈발집합 찾기 : Apriori 알고리즘] 아이템의 연관 관계를 빠르게 파악하는 알고리즘인 Apriori 알고리즘에 대해 배워보고, MLEXTEND를 활용해 실습해보도록 하겠습니다.


In [ ]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import get_file
import scipy

In [ ]:
# 영화 포스터를 가져와 주피터에서 볼 수 있도록 만들어주는 메소드
def display_poster(if_item, then_item):
    import requests
    from io import BytesIO
    from PIL import Image
    
    def get_poster(movie_id):
        url = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/img/POSTER_20M_FULL/{}.jpg".format(movie_id)
        try:
            response = requests.get(url)
            b = BytesIO(response.content)
            img = np.asarray(Image.open(b))
        except:
            img = np.zeros((200,100,3))
        return img
    
    def get_movie_title(movie_id):
        global movie_df
        return movie_df.loc[movie_df.movie_id==movie_id,'title'].iloc[0]
    
    if_image = get_poster(list(if_item)[0])
    if_title = get_movie_title(list(if_item)[0])
    then_image = get_poster(list(then_item)[0])
    then_title = get_movie_title(list(then_item)[0])

    fig = plt.figure(figsize=(8,8))
    fig.set_size_inches((20,5))    
    
    ax = fig.add_subplot(1,2,1)
    ax.set_title(f'[antecedent]{if_title}')
    ax.imshow(if_image)

    ax = fig.add_subplot(1,2,2)
    ax.set_title(f'[consequent]{then_title}')
    ax.imshow(then_image)    
    plt.tight_layout()
    plt.show()    

# \[ 아이템 간 관계를 나타내는 여러가지 지표 \]
----

## 1. 연관분석이란? 
연관분석(Association Analysis)은 대용량의 거래(transaction) 데이터로부터 "X를 구매했으면, Y를 구매할 것이다" 형식의 아이템 간 연관 관계를 분석하는 방법입니다. <br>
보통 **장바구니 분석(Market Basket Analysis)**로 불리기도 합니다. 즉, 고객의 장바구니에 어떤 아이템이 동시 담겼는지 패턴을 파악하여 상품을 추천하는 방법입니다. <br>

<img src = 'https://i.imgur.com/ROqlQxD.png'>

## 2. 연관 분석의 주요 지표

연관분석에서는 크게 지지도, 신뢰도, 리프트라는 세 가지 지표를 통해 아이템 간의 관계를 표현합니다. 각각의 의미를 알아봅시다. 

### (1) Support

스타워즈2를 재미있게 본 유저가 있다고 합시다. 이 유저에게 어떤 영화를 추천하는 것이 좋을까요? 가장 단순한 방법은 각각의 영화를 전체 유저 중에 얼마나 되는 사람이 좋아하는지 알아보고, 많은 인기(혹은 지지)를 받은 영화를 찾아서 추천하는 것입니다. 예를 들면 대부분 사람들이 타이타닉을 선호하는 만큼 해당 유저도 타이타닉을 선호할 거으로 보는 것이죠. <br>
이 확률값을 '지지도(Support)'라고 부릅니다. 전체 유저 중에 스타워즈 3, 스타트렉, 러브액츄얼리, 타이타닉을 선호하는 유저의 수를 각각 구하면 알 수 있습니다.<br>
<br>
![](https://i.imgur.com/NMjCtIJ.jpg)


### (2) Confidence

"유저가 스타워즈2를 재미있게 보았다"는 정보를 이용해서 영화 스타워즈3를 좋아할 확률을 보다 정확하게 알 수는 없을까요? 스타워즈2를 좋아하는 유저 중에는 대상 영화를 좋아하는 유저가 얼마나 되는지 알아볼 수 있을 것입니다. 스타워즈2를 좋아하는 유저들 중에 대상 영화를 좋아했던 유저가 많다면, 이 유저 역시 대상 영화를 좋아할 확률이 높다고 보는 것이죠. 이 확률값을 '신뢰도(Confidence)'라고 하며, 영화X를 좋아하는 유저 중에 영화Y를 좋아하는 유저(즉, 영화X와 영화Y를 모두 좋아한 유저)의 비율로 계산합니다. <br>

![](https://i.imgur.com/fRF3mK4.jpg)

### (3) Lift

그렇다면 스타워즈2를 선호했다는 사실이 대상 영화 대한 선호를 파악하는데 얼마나 중요했을까요?
유저 전반적으로 대상 영화 Y를 좋아할 확률(지지도)보다 스타워즈2라는 영화를 좋아하는 사람 중에 대상 영화 Y를 좋아할 확률(신뢰도)이 더 크다면, 스타워즈2를 선호한다는 사실이 대상 영화Y를 선호할 것으로 예상하는 데에 대한 확신을 높여줄 것입니다. 
<br><br>
$$\text{confidence}(\text{StarWars2} \rightarrow \text{StarWars3}) > \text{support}(\text{StarWars3}) $$
<br>
반면에, 전반적으로 타이타닉을 좋아할 확률이 스타워즈2를 좋아하는 사람 중에 타이타닉을 좋아할 확률(신뢰도)이 더 높다면,
타이타닉과 스타워즈2의 연관관계는 높지 않을 것입니다.
<br>
$$\text{confidence}(\text{StarWars2} \rightarrow \text{Titanic}) < \text{support}(\text{Titanic}) $$
<br>
이처럼 지지도와 신뢰도를 이용해 아이템의 관계를 파악하는 지표가 바로 리프트(Lift)입니다. 리프트는 어떻게 구할까요? 
<br><br>
$$\text{lift}(\text{StarWars2} \rightarrow Y) = \frac{\text{confidence}(\text{StarWars2} \rightarrow Y)}{\text{support(Y)}}$$
<br>
리프트가 1보다 크면 전자의 상황을, 1보다 작으면 후자의 상황을 뜻하는 것이죠. 스타워즈2를 재미있게 보았다는 정보를 얻고 나니 대상 영화Y를 재미있게 볼 확률이 기본 확률값(지지도)에 비해 높아졌는지, 낮아졌는지 확인하는 것이죠. '리프트'라는 지표의 이름은 "어떤 증거가 신뢰도를 높여주는가?"라는 의미에서 나온 것입니다.

![](https://i.imgur.com/3qs1EyN.jpg)

# [ 연관분석 수행하기 ] 
---

## 1. 데이터 가져오기


In [ ]:
%%time
ROOT_URL = "https://movielens.cjkizsqpfbi3.ap-northeast-2.rds.amazonaws.com/recommender_systems/movielens/datasets"

movie_path = get_file("movies.csv", os.path.join(ROOT_URL, "movies.csv"))
movie_df = pd.read_csv(movie_path)

genre_path = get_file("genres.csv", os.path.join(ROOT_URL, "genres.csv"))
genre_df = pd.read_csv(genre_path)

rating_path = get_file("ratings.csv", os.path.join(ROOT_URL, "ratings.csv"))
rating_df = pd.read_csv(rating_path)

## 2. 데이터 전처리하기

### (1) 평점 정보 중에서 선호 영화 추려내기
`rating_df`에는 각 유저가 영화마다 평점을 매긴 정보가 있습니다. 이 데이터를 그대로 사용하면 될까요? 아닙니다. 비슷한 영화를 찾으려면, 어떤 유저가 '영화를 보았다'가 아니라 '영화를 보고 만족했다'를 가지고 관계도를 그려야 할 것입니다. 유저가 평점을 줬다고 해서 만족한 것은 아닐 것이기 때문이죠. <br>

#### 4점 이상의 평점 데이터 가져오기 

평점 정보는 0.5~5.0점까지 0.5점 단위로 구성되어 있습니다. 5점일수록 영화를 긍정적으로 평가한 경우고, 0점에 가까울수록 낮게 평가한 경우입니다. 이번에는 4점 이상 영화를 평가한 경우를 긍정적으로 평가한 것으로 파악하고, 4점 이상 영화를 평가한 경우만을 뽑아내도록 하겠습니다. 

In [ ]:
# 평점 데이터에서 고유한 값들을 가져오자
unique_ratings

In [ ]:
# 평점 데이터 중에서 4점이상 평가를 높게 준 것들만 추리자
over_4_rating_df

### (2) 영화 정보를 <장바구니> 단위로 나누기

앞서 언급하였듯이, 영화는 마트에서 물건를 사는 것과 다르게 동시에 두 영화를 틀어놓고 볼 수는 없습니다. 때문에 각 유저가 긍정적으로 평가한 영화들을 하나의 장바구니로 해석합니다.

#### 유저 별로 긍정적으로 평가한 영화 집합(set) 가져오기

In [ ]:
# 유저 별로 묶은 후, 각 유저 별 평점을 내린 영화 아이디에 대해 일괄적으로 하나의 집합set으로 구성해주자
baskets_series.head(5)

#### 총 장바구니 수 가져오기

총 유저의 수(=장바구니의 갯수)와 동일합니다. 각 장바구니는 한 유저가 동시에 긍정적으로 평가한 영화들을 묶어둔 것입니다.

In [ ]:
len(baskets_series)

#### c.f) Set에서의 부등호 연산

set은 부등호 연산을 지원하는데, 부등호 기호를 통해 포함관계를 파악할 수 있습니다.<br>

In [ ]:
a = set([1,2,3,4])
b = set([1,2])

print("b는 a에 포함되어 있다  -> ", a>=b)
print("a는 b에 포함되어 있다  -> ", a<=b)

#### 특정 영화를 본 유저(장바구니)를 가져오기

set의 부등호 연산을 이용해 특정 영화를 본 유저를 가져올 수 있습니다.

In [ ]:
# 1번 영화를 본 장바구니(유저) 가져오기

In [ ]:
# 1,5번 영화를 동시에 본 장바구니(유저) 가져오기

### (3) 장바구니에 영화가 있는지 확인하기

장바구니 분석을 하기 위해서는 장바구니에 물건의 조합(pair)이 담겨있는지를 확인해야 합니다. 영화 추천에서는 어떤 영화 조합(pair)를 모두 본 유저가 있는지 확인해야 할 것입니다. <br>

#### 아래 5개 영화를 모두 본 유저 가져오기

In [ ]:
movie_df.columns = ['movie_id', 'title', 'genres']

In [ ]:
# movie_df에서 확인하기
items = [5378, 33493, 6942, 1721, 68358]

In [ ]:
# items가 모두 포함한 장바구니만을 선택


#### `check_in_baskets` 함수 포함하기 
5개가 모두 담긴 장바구니, 즉 5개를 모두 좋게 평가한 유저를 찾을 수 있습니다.

In [ ]:
def check_in_baskets(items, baskets_series):
    """
    baskets 별로 movie id로 이루어진 items가 모두 포함되어 있는지를 
    확인하여 반환하는 메소드
    """
    return baskets_series >= items

## 2. 연관분석 평가척도 계산하기


### (1) 지지도 (Support)
$$
S(X) = \frac{Freq(X)}{N}  \\
S(X, Y) = \frac{Freq(X,Y)}{N} 
$$

일반적으로 장바구니 분석에서 지지도(Support)는 전체 거래 횟수에서 아이템을 포함하는 거래 횟수를 나누어준 값입니다. 지지도를 단일 아이템에 대해 계산하면 이 아이템의 거래가 얼마나 자주 이루어졌는지 평가해줍니다. 한편 두 아이템의 조합(pair)에 대한 지지도를 계산하면, 두 아이템이 동시에 팔린 거래가 얼마나 되는지 알 수 있습니다. 

In [ ]:
def calculate_support(baskets_series, X):

    return 

먼저 단일 영화에 대한 지지도를 살펴봅시다. 

In [ ]:
support = calculate_support(baskets_series, 5378)
print(f"스타워즈 2가 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, 33493)
print(f"스타워즈 3가 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, 68358)
print(f"스타트렉이 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, 6942)
print(f"러브 액츄얼리가 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, 1721)
print(f"타이타닉이 포함된 바스켓의 비율 : {support:.3%}")

전체 유저중에 타이타닉을 좋게 평가한 유저의 비율이 10% 정도로 가장 많은 것을 알 수 있습니다. 

In [ ]:
support = calculate_support(baskets_series, [5378, 33493])
print(f"스타워즈 2와 스타워즈 3가 동시에 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, [5378, 68358])
print(f"스타워즈 2와 스타트렉이 동시에 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, [5378, 6942])
print(f"스타워즈 2와 러브 액츄얼리가 동시에 포함된 바스켓의 비율 : {support:.3%}")

support = calculate_support(baskets_series, [5378, 1721])
print(f"스타워즈 2와 타이타닉이 동시에 포함된 바스켓의 비율 : {support:.3%}")

스타워즈2와 스타워즈3를 모두 좋게 평가한 유저는 전체 유저의 1.884%인 반면, 스타워즈2와 러브 액츄얼리를 모두 좋게 평가한 유저는 전체 유저의 0.492%입니다. 

### (2) 신뢰도 (Confidence)
$$C(X \rightarrow Y) = \frac{Freq(X,Y)}{Freq(X)}$$

장바구니 분석에서 신뢰도는 아이템 X를 구매한 장바구니 수로 아이템 Y와 X를 함께 구매한 장바구니 수를 나누어준 값입니다. 이는 X를 구매했을 때, Y를 얼마나 많이 구매하는지를 평가하는 지표입니다.

위 수식은 사실 2개의 지지도 정보로 구할 수 있습니다.

$$C(X \rightarrow Y) = \frac{Freq(X,Y)}{Freq(X)} = \frac{Freq(X,Y)/N}{Freq(X)/N} = \frac{S(X,Y)}{S(X)}
$$
지지도를 이용해 신뢰도를 계산해 보도록 하겠습니다.

In [ ]:
def calculate_confidence(baskets_series, X, Y):

    return

In [ ]:
confidence = calculate_confidence(baskets_series, 5378, 33493)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 스타워즈 3를 긍정적으로 평가할 확률 : {confidence:.3%}")

confidence = calculate_confidence(baskets_series, 5378, 68358)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 스타트렉를 긍정적으로 평가할 확률 : {confidence:.3%}")

confidence = calculate_confidence(baskets_series, 5378, 6942)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 러브 액츄얼리를 긍정적으로 평가할 확률 : {confidence:.3%}")

confidence = calculate_confidence(baskets_series, 5378, 1721)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 타이타닉을 긍정적으로 평가할 확률 : {confidence:.3%}")

결과를 살펴보겠습니다. 지지도에서는 조합간에 수치 차이가 크지 않았다면, 신뢰도의 경우에는 스타워즈3와의 지표가 다른 조합과 큰 차이로 높게 나타남을 알 수 있습니다. 

### (3) 리프트 (Lift)

$$
L(X\rightarrow Y) = \frac{C(X \rightarrow Y)}{S(Y)}
$$

장바구니 분석에서 리프트는 Y를 구매한 사람의 비율($S(Y)$)보다 X를 구매한 사람 중 Y도 구매한 사람의 비율($C(X \rightarrow Y)$)이 높다면,즉 리프트가 1보다 크다면($L(X\rightarrow Y)>1$) X와 Y의 관계 강도가 강하다고 봅니다.<br>

In [ ]:
def calculate_lift(baskets_series, X, Y):
    
    return

In [ ]:
lift = calculate_lift(baskets_series, 5378, 33493)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 스타워즈 3를 긍정적으로 평가하는 것에 대한 리프트 : {lift:.3f}")

lift = calculate_lift(baskets_series, 5378, 68358)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 스타트렉를 긍정적으로 평가하는 것에 대한 리프트 : {lift:.3f}")

lift = calculate_lift(baskets_series, 5378, 6942)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 러브 액츄얼리를 긍정적으로 평가하는 것에 대한 리프트 : {lift:.3f}")

lift = calculate_lift(baskets_series, 5378, 1721)
print(f"스타워즈 2를 긍정적으로 평가한 사람이 타이타닉을 긍정적으로 평가하는 것에 대한 리프트 : {lift:.3f}")

이를 통해 우리는 스타워즈 2는 같은 시리즈 물인 스타워즈과 강한 연관관계가 있음을 보여주고, 비슷한 SF 장르인 스타트렉과도 연관관계가 있음을 알 수 있습니다.<br> 
그와 달리 로맨스물인 러브 액츄얼리와 타이타닉와는 낮은 연관관계에 있는 것으로 나왔습니다. <br>
<br>

### (4) 세 지표의 활용법

연관분석에서 실제 지지도(Support)와 신뢰도(Confidence), 리프트(Lift) 세 가지 지표는 어떤 식으로 활용될까요? 연관 분석을 통한 연관 추천 알고리즘 순서는 아래와 같습니다.
<br>
<br>
![](https://i.imgur.com/vRyh5ya.jpg)
<br>
<br>
먼저, 지지도와 신뢰도의 최소 기준을 정하여 최소 기준에 미달하는 연관 관계를 제거해 나갑니다. 지지도가 너무 작으면 리프트 값이 실제 의미보다 과하게 나올 수 있게 때문에, 걸러주는 것이 좋습니다. 그 다음에는 리프트을 기준으로 각 아이템과 연관 관계가 강한 아이템을 순서대로 보여주는 것이죠. 이제 이 세가지 지표를 적용해 각 아이템 쌍에 대한 연관 강도를 구하고, 추천할 아이템을 추려내는 연관분석 추천 시스템을 만들어봅시다.

# \[ 빈발집합 찾기 : Apriori 알고리즘과 FPGrowth 알고리즘 \]
---

빈번하게 등장한 아이템의 쌍을 **빈발집합**이라고 부릅니다. 앞서 연관분석의 세 가지 주요 지표의 수식을 떠올려보면, 모두 빈도(`Freq()`)를 이용해 만들어졌음을 알 수 있습니다. 연관분석을 실제 구매 데이터에 적용한다면, 각각의 아이템의 쌍이 얼마나 등장했는지를 세어야 할 것입니다.하지만 아이템의 가짓수가 늘어나고, 확인해야 할 바스켓의 수가 커지면, 이에 대한 계산은 기하급수적으로 늘어나게 됩니다. 
<br>
<br>
이 문제를 해결하기 위해 고안된 것이 자주 등장하는 아이템의 쌍만을 빠르게 추려 계산하는 **빈발집합 탐색 알고리즘**입니다.
대표적인 빈발집합 탐색 알고리즘으로는 Apriori 알고리즘과 FP-Growth 알고리즘이 있습니다.
둘 다 데이터 셋 내에서 빈발집합을 찾아내고, 몇 번이나 등장했는지를 세어주는 알고리즘으로, 두 알고리즘의 결과는 동일합니다.
코드의 최적화 수준에 따라 조금씩 달라지지만, 일반적으로 FP-Growth 알고리즘이 Apriori 알고리즘보다 빠릅니다.
<br>
<br>
이번에는 Apriori 알고리즘을 사용하겠습니다. Apriori 알고리즘은 모든 가능한 조합의 개수를 줄이는 전략을 사용합니다.아래 이미지를 보면, 5가지 아이템이 있다고 할 때, 이 5가지를 이용해 나올 수 있는 가능한 조합은 총 $2^5 -1 = 31$개 입니다.아이템 수가 늘어날수록 아이템 조합 역시 급격하게 늘어날 것입니다. 
<br>
<br>
Apriori는 각 조합의 지지도를 구하면서 조합의 아이템 수를 늘리며 내려가면서 어떤 조합의 지지도가 일정 기준 이하로 떨어지면, 그 아래로 내려가도(즉, 조합의 아이템 수를 늘리더라도) 빈발집합이라고 볼 수 없다 판단하여 더 이상 가지를 따라 내려가지 않고 쳐내는 식으로 빈발집합을 탐색합니다.

<img src = 'https://i.imgur.com/pZ75IjW.png'>

In [ ]:
# 빈발집합 알고리즘에 대한 simple code 
from tqdm import tqdm
from itertools import combinations

# parameter setting


# 전체 아이템 셋

results = {}
for i in range(1, max_len+1):
    # 후보군 가져오기
    
    # candidate별 support 구하기

    # min support 보다 높은 빈발집합을 결과에 담기
    
    # min_support보다 높은 빈발집합 케이스 가져오기
    
    # min_support 기준보다 높은 것들끼리 self_join을 통해 다음 후보군을 구성

## 1. 장바구니 정보를 One-hot Encoding하기

#### `mlxtend` 가져오기

In [ ]:
# Ipython에서 지원하는 기능으로, 이를 통해 필요한 라이브러리를 설치할 수 있습니다.
# !pip install mlxtend 
import mlxtend
from mlxtend.preprocessing import TransactionEncoder

#### TranscationEncoder 동작 확인하기

In [ ]:
transaction_encoder = TransactionEncoder()

transaction_encoder.fit_transform([
    {0,1,2},
    {0,1,4},
    {1,5},
    {3,4,5},
    {2,3}])

#### `TranscationEncoder`를 통해 장바구니 정보를 Multi-Hot 벡터로 만들기

In [ ]:
# 바스켓을 하나의 벡터로 표현하기


#### 바스켓 Multi-hot 행렬을 pd.Dataframe으로 변환하기

`np.array`로 만들어진 `baskets_array`를 이용해 `pd.DataFrame()`을 만들어줍니다. 이 데이터의 칼럼은 영화를 나타내므로, `movie_id`를 받아와 살럼명으로 지정해줍니다. 

In [ ]:
# 컬럼 이름을 movie_id로 바꾸기

baskets_df.head(5)

#### 평점 갯수 상위 5000개의 영화만 추려내기

전체 데이터를 통해 빈발집합 찾기 알고리즘을 수행하면, 시간이 너무 많이 걸리기 때문에, 상위 5000개의 영화에 한해서 빈발 집합을 찾도록 하겠습니다. 평점 수 상위 5000개의 영화를 추려봅시다.

In [ ]:
# 평점 데이터 중에서 영화 아이디를 기준으로 평점의 개수를 세고
# 평점 개수를 기준으로 상위 5000개의 영화 아이디를 가져오자
selected_basket_df.head()

top 5000 영화를 한 편도 보지 않은 장바구니(유저) 정보는 필요하지 않습니다. 각 행(각 유저의 장바구니)의 평가 여부를 확인하여 top5000영화에 대한 평가가 없는 장바구니는 제거합니다. 

In [ ]:
# 5000개 영화중 하나도 보지 않은 바스켓을 제거하자

## 2. Apriori를 통해 원하는 연관규칙 쌍을 추출하기

MLxtend에서는 apriori 알고리즘으로 아래와 같이 지원합니다. min support 값은 데이터마다 다르게 지정해줍니다. 

In [ ]:
# apriori 알고리즘
from mlxtend.frequent_patterns import apriori

freq_sets_df = apriori(selected_basket_df.sample(frac=0.05),
                       min_support=0.01, # 1% 이상 포함된 경우만
                       max_len=2, # 빈발집합의 최대 크기
                       use_colnames=True,
                       verbose=1)

위를 실행시키려면 최소 30기가 이상의 램 메모리를 필요로 합니다. 서버 컴퓨터를 이용해 사전에 처리한 결과를 아래를 통해 받을 수 있습니다.

In [ ]:
# 처리한 결과 파일 가져오기
fqs_path = get_file("frequent_sets.pkl",
                    'https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/results/frequent_sets.pkl')
freq_sets_df = pd.read_pickle(fqs_path)
freq_sets_df

각 빈발 집합 별 지지도가 계산되어 있습니다. 우리는 지지도 값 만으로 아래 관계식을 통해, 우리가 궁금한 세가지 지표(지지도, 신뢰도, 리프트)를 모두 도출할 수 있습니다. 

$$
S(X) = \frac{Freq(X)}{N}  \\
S(X, Y) = \frac{Freq(X,Y)}{N}  \\
C(X \rightarrow Y) = \frac{S(X,Y)}{S(X)} \\
L(X \rightarrow Y) = \frac{C(X \rightarrow Y)}{S(Y)}
$$


## 3.  연관분석 진행하기

MLxtend에서는 연관 관계를 파악하는 데 필요한 세 가지 지표 (Support, Confidence, Lift)를 바로 계산해주는 `association_rules`를 제공합니다. Apriori로 도출된 빈발집합 정보를 `association_rules`에 넣어주면 우리가 원하는 아이템 간 연관관계를 파악할 수 있습니다.

#### `association_rules` 가져오기

In [ ]:
from mlxtend.frequent_patterns import association_rules # 연관분석

#### 연관관계 추려내기

* support > 0.01
* confidence > 0.1

In [ ]:
# Support과 Confidence로 유의미한 연관관계 추리기
item_rules = association_rules(freq_sets_df, 
                               metric='support',
                               min_threshold=0.01)  
item_rules = item_rules[item_rules.confidence > 0.1]

In [ ]:
item_rules

#### 리프트 기준으로 데이터 정렬하기

In [ ]:
# 리프트 값으로 정렬하기
item_rules = item_rules.sort_values('lift', ascending=False)
item_rules.head(20)

#### 결과 시각화하기 


In [ ]:
for _, row in item_rules.head(20).iterrows():
    display_poster(row.antecedents, row.consequents)

In [ ]:
# item_ruls의 antecedents와 consequents의 movie_id를 영화 제목을 바꿔보자.
# 영화 아이디가 키값이고, 영화 제목이 밸류값인 딕셔너리형를 이용

# set에 있는 영화 아이디 값 하나를 가져와 영화 제목으로 바꾸어줌


In [ ]:
item_rules.head(10)

"해리포터" 영화와 강한 연관관계를 가지는 영화 리스트 상위 5개는 아래와 같습니다. 대부분이 해리포터 시리즈로 나타나게 됩니다.

In [ ]:
item_rules[item_rules.antecedents == 'Harry Potter and the Deathly Hallows: Part 1'].iloc[:5]

"메멘토"이라는 영화와 연관관계를 가지는 영화 리스트들은 아래와 같습니다.

In [ ]:
item_rules[item_rules.antecedents == 'Memento'].iloc[:5]

"인크레더블"이라는 영화와 연관관계를 가지는 영화 리스트들은 아래와 같습니다. 

In [ ]:
item_rules[item_rules.antecedents == 'Incredibles, The'].iloc[:5]

### [ Additional ] Youtube 추천시스템에서의 연관 분석


<img src="https://i.imgur.com/CfgM7U8.png" width="500">

연관 분석을 이용한 추천시스템은 간단해보이지만, 커머스 서비스나 컨텐츠 서비스에서 가장 일반적으로 사용되고 있는 추천 방식입니다. 가장 대표적인 추천 시스템인 Youtube도 2010년에 제시한 [Youtube 추천 시스템](https://www.inf.unibz.it/~ricci/ISR/papers/p293-davidson.pdf) 구성에 따르면 추천 후보군을 선정할 때 연관분석을 통해, 높은 상관관계를 가진 컨텐츠를 추려내는 방식으로 추천 후보군 구성했다고 합니다. 그리고 추려진 추천 후보군 중에서 고객의 정보를 통해 추천에 대한 우선순위를 매겨 배치하는 방식을 택했습니다.  단순해보이지만, 이 방식을 이용해 유튜브는 자사 서비스의 클릭률(CTR)을 거의 두배 가량 높습니다(아래 그래프 참고). 당시 대략 60%의 비디오 클릭이 홈 화면에 노출되는 추천 영상에 의해 이루어졌다고 합니다.

#  

---

    Copyright(c) 2020 by Public AI. All rights reserved.
    Writen by PAI, SeonYoul Choi ( best10@publicai.co.kr )  last updated on 2020/05/30


---